***
# Python Alchemy - Volume One
# Chapter 11 - Mastering Python Functions

- 11.1 Advanced Argument Handling
- 11.2 Default Arguments and Common Pitfalls
- 11.3 Higher-Order Functions
- 11.4 Closours
- 11.5 Function Decorators
- 11.6 Understanding Python Iterators
- 11.7 Understanding Python Generators
- 11.8 Function Introspection & Metadata
- 11.9 Functional Programming Paradigm
- 11.10 Functional Approach to Data Transformation
- 11.11 Functools

***

## 11.1 Advanced Argument Handling

To give developers more control, Python provides positionalonly and keyword-only parameters.

For example, a function handling geometry might require radius as a positional-only argument while requiring color and fill as keyword-only arguments for clarity.

In [ ]:
def draw_circle(radius, /, *, color="black", fill=False):
    print(f"Drawing a circle of radius {radius}, color={color}, fill={fill}")
    # Must pass radius as positional

draw_circle(10, color="red", fill=True) # Works
draw_circle(radius=10, color="red") # Error: radius is positional-only

Here, / ensures radius is positional-only, while * makes color and fill keyword-only. This prevents misuse and keeps the function’s usage crystal clear.

Let’s take another code example to get more clarity:

In [ ]:
def process_payment(amount, currency, /, *, method="card", receipt_email=None):
    """
    Process a payment with specific rules:
    - amount and currency must always be positional (enforced with ‘/’)
    - method and receipt_email must always be keyword arguments(enforced with ‘*’)
    """

    print(f"Processing {amount} {currency} using {method}")
    if receipt_email:
        print(f"Sending receipt to {receipt_email}")
    else:
        print("No receipt email provided.")


# Correct usage
process_payment(100, "USD", method="paypal", receipt_email="user@example.com")
# Positional required, keyword-only required
process_payment(50, "EUR", method="card")


Processing 100 USD using paypal
Sending receipt to user@example.com
Processing 50 EUR using card
No receipt email provided.


In [ ]:
# Error: currency cannot be passed as a keyword argument
process_payment(amount=200, currency="INR", method="upi")

TypeError: process_payment() got some positional-only arguments passed as keyword arguments: 'amount, currency'

In [3]:
# Error: method cannot be passed without keyword
process_payment(300, "GBP", "cash")

TypeError: process_payment() takes 2 positional arguments but 3 were given

## 11.3 Higher-Order Functions

A higher-order function is a function that either accepts another function as an argument or returns a new function as its output.

Let’s take a code example, Python’s built-in map() and filter() are classic higher-order functions, they take another function and apply it to every element in a sequence:

In [ ]:
numbers = [1, 2, 3, 4]
squared = map(lambda x: x**2, numbers)

Here, the lambda function is passed into map() to process each number. Similarly, you can design your own higher-order functions, like one that returns a custom multiplier:

In [4]:
def make_multiplier(factor):
    def multiply(x):
        return x * factor
    return multiply

double = make_multiplier(2)
print(double(5)) # Output: 10

10


## 11.4 Closours

In Python, nested functions (functions defined inside other functions) offer a powerful way to create small, self-contained units of logic that are closely tied to their surrounding context. When an inner function references variables from its outer function, it forms what’s known as a closure.

For example:

In [5]:
def make_counter():
    count = 0 # This variable belongs to the outer function
    
    def counter():
        nonlocal count # Allows modification of the outer variable
        count += 1
        return count
    
    return counter # Returning the inner function (with memory attached)

# Create two independent counters
counter1 = make_counter()
counter2 = make_counter()

print(counter1()) # Output: 1
print(counter1()) # Output: 2
print(counter2()) # Output: 1

1
2
1


## 11.5 Function Decorators

Decorators in Python are like gift wrappers for your functions, they let you enhance or modify a function’s behavior without permanently changing its original structure.

A basic decorator can be written by defining a function that accepts another function, defines an inner wrapper function, and then returns that wrapper. The wrapper usually calls the original function while adding something extra before or after it runs.

In [6]:
def greet_decorator(func):
    def wrapper():
        print("Hello! Welcome to Python World.")
        func()
        print("Goodbye!")
    return wrapper

@greet_decorator
def say_hello():
    print("I'm learning decorators.")

say_hello()

Hello! Welcome to Python World.
I'm learning decorators.
Goodbye!


say_hello() → wrapper() → func() → print(“I’m learning decorators.”)

#### Decorating Functions with Arguments

When a decorated function accepts its own parameters, the decorator must be able to forward these arguments seamlessly while preserving the original function’s behavior.

Suppose we want a decorator that measures the execution time of any function, regardless of the number or type of arguments it accepts:

In [7]:
import time

def timer(func):
    # Wrapper function that can handle any arguments
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs) # Call the original function with parameters
        end_time = time.time()
        print(f"Function '{func.__name__}' executed in {end_time - start_time:.6f} seconds")
        return result
    return wrapper

# Decorating a function with parameters
@timer
def compute_power(base, exponent):
    time.sleep(0.5) # Simulate a delay
    return base ** exponent

# Calling the decorated function
result = compute_power(5, 3)
print("Result:", result)

Function 'compute_power' executed in 0.500157 seconds
Result: 125


#### Understanding Multiple Decorators

When you use multiple decorators on a single function, Python allows you to layer functionality, in such case each decorator wraps the function (or another decorator) to add its own behavior.

Let’s demonstrate with a practical example that shows how decorators stack and execute in order:

In [8]:
def bold(func):
    def wrapper():
        print("<b>", end="")
        func()
        print("</b>", end="")
    return wrapper


def italic(func):
    def wrapper():
        print("<i>", end="")
        func()
        print("</i>", end="")
    return wrapper

@bold
@italic
def greet():
    print("Hello, Python!", end="")

    
greet() # Output: <b><i>Hello, Python!</i></b>

<b><i>Hello, Python!</i></b>

When Python reaches the decorated function, it interprets it as:
greet = bold(italic(greet))

#### Decaroters with Arguments

Typically, a decorator in Python accepts a single argument, that is, the function it is intended to enhance. However, there are scenarios where it is desirable for the decorator itself to be configurable or parameterized, allowing it to modify its behavior based on external inputs. For example:

In [9]:
def log_message(level):
    # Outer function: captures the parameter ‘level’
    def decorator(func):
        # Middle function: accepts the target function
        def wrapper(*args, **kwargs):
            # Innermost function: defines additional behavior
            print(f"[{level}] Executing '{func.__name__}'")
            result = func(*args, **kwargs)
            print(f"[{level}] Completed '{func.__name__}'")
            return result
        return wrapper
    return decorator

# Applying the parameterized decorator
@log_message("INFO")
def greet(name):
    print(f"Hello, {name}!")

@log_message("ERROR")
def divide(a, b):
    return a / b

greet("Ivaan")
divide(10, 2)

[INFO] Executing 'greet'
Hello, Ivaan!
[INFO] Completed 'greet'
[ERROR] Executing 'divide'
[ERROR] Completed 'divide'


5.0

## 11.6 Understanding Python Iterators

Iterators are objects that allow you to traverse through a sequence of data one element at a time without needing to store the entire sequence in memory. They represent a powerful abstraction of the iteration process, letting you access elements lazily (only when needed) instead of all at once.

When you write a simple for loop like:

In [10]:
for num in [1, 2, 3]:
    print(num)

1
2
3


Internally, the loop above behaves roughly like this:

In [11]:
numbers = [1, 2, 3]
it = iter(numbers)

while True:
    try:
        num = next(it)
        print(num)
    except StopIteration:
        break

1
2
3


This lazy, step-by-step approach makes iteration memory-efficient and elegant especially when working with large datasets or streams of data.

#### iter() and next() in Action

The built-in iter() function in Python transforms an iterable object (such as a list, tuple, or string) into an iterator, a specialized object that maintains state and produces elements one at a time upon request. Let’s take another example:

In [12]:
fruits = ["apple", "banana", "cherry"]
it = iter(fruits)
print(next(it)) # apple
print(next(it)) # banana
print(next(it)) # cherry

apple
banana
cherry


When an iterator has no remaining elements to yield, invoking next(it) raises a StopIteration exception

#### Writing a custom countdown Ittrator

In Python, it is entirely possible to construct custom iterators by implementing the language’s built-in dunder (double underscore) methods, which define the iterator protocol. For example:

In [14]:
class Countdown:
    def __init__(self, start):
        self.current = start
    def __iter__(self):
        return self
    def __next__(self):
        if self.current <= 0:
            raise StopIteration
        self.current -= 1
        return self.current + 1
    
for number in Countdown(5):
    print(number)

5
4
3
2
1


Here, Countdown acts as both an iterable and an iterator. Each call to next() moves the internal state forward, returning one item at a time until the countdown reaches zero.

## 11.7 Understanding Python Generators

Generators in Python are a special kind of function designed to produce values one at a time in a lazy and memory-efficient way.

Let’s understand this with a simple example function introducing yield:

In [15]:
def countdown(n):
    while n > 0:
        yield n
        n -= 1

for number in countdown(5):
    print(number)

5
4
3
2
1


Here, countdown() doesn’t create a list of numbers. Instead, it yields one value at a time, pausing after each yield until the next loop iteration asks for more.

#### Generators: Under the Hood

Under the hood, when Python encounters a function containing a yield statement, it transforms that function into a generator factory: a construct designed to produce generator objects rather than execute the function body immediately.

Let’s use an example to visualize what’s happening internally:

In [16]:
def simple_generator():
    print("Step 1: Start")
    yield 10
    print("Step 2: Resume after first yield")
    yield 20
    print("Step 3: Resume after second yield")
    yield 30
    print("Step 4: End of generator")

gen = simple_generator()
print("Generator created.")
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen)) # Rsise StopIteration

Generator created.
Step 1: Start
10
Step 2: Resume after first yield
20
Step 3: Resume after second yield
30
Step 4: End of generator


StopIteration: 

Here. When Python sees def simple_generator():, it marks it as a generator function (because of yield), not a regular function.

#### Generator Special Methods

Python generators come equipped with several special methods: send(), throw(), and close(), these methods extend their functionality beyond simple iteration, allowing for more interactive and controlled execution.

#### .send() Method

The .send() method in Python’s generator protocol allows two-way communication between a generator and its caller, turning generators into interactive and stateful components rather than simple value producers.

Let’s understand this with the following example:

In [17]:
def interactive_counter():
    count = 0
    while True:
        increment = yield count # yield acts as an expression
        if increment is None: # If nothing sent, default increment
            increment = 1
        count += increment


# Create the generator
counter = interactive_counter()
print(next(counter)) # Start generator → yields 0
print(counter.send(5)) # Send 5 → adds 5 → yields 5
print(counter.send(3)) # Send 3 → adds 3 → yields 8
print(next(counter)) # Resume without sending → adds 1 → yields 9

0
5
8
9


In this example, the generator begins with count = 0 and pauses at the first yield. When .send(5) is called, the value 5 is injected into the generator, becoming the result of the suspended yield expression.

#### .threw() Method

The .throw() method in Python’s generator protocol allows an exception to be injected directly into a generator while it is running.

Let’s extend our interactive counter example to see how it works:

In [18]:
def interactive_counter():
    count = 0
    try:
        while True:
            increment = yield count
            if increment is None:
                increment = 1
            count += increment
    except ValueError:
        print("Invalid input detected — resetting counter.")
        count = 0
        yield count


# Create generator
counter = interactive_counter()
print(next(counter)) # Starts generator → 0
print(counter.send(5)) # Adds 5 → 5
print(counter.send(3)) # Adds 3 → 8
print(counter.throw(ValueError)) # Triggers exception inside generator → handled → 0
print(next(counter)) # Continues normally → 1

0
5
8
Invalid input detected — resetting counter.
0


StopIteration: 

In this example, the .throw(ValueError) call raises a ValueError inside the generator at the point of suspension.

#### .colse() Method

The .close() method in Python’s generator protocol provides a clean and controlled way to stop a generator. For example:

In [19]:
def interactive_counter():
    count = 0
    try:
        while True:
            increment = yield count
            if increment is None:
                increment = 1
            count += increment
    except GeneratorExit:
        print("Generator closed. Performing cleanup...")


# Create the generator
counter = interactive_counter()
print(next(counter)) # Start → yields 0
print(counter.send(5)) # Send 5 → yields 5
print(counter.send(3)) # Send 3 → yields 8
counter.close() # Gracefully close the generator
print("Generator has been closed.")

0
5
8
Generator closed. Performing cleanup...
Generator has been closed.


In this illustration, the generator executes normally until .close() is called.

#### Chaining Generators - Building Data Pipelines

Chaining generators is one of Python’s most elegant and powerful techniques for building data pipelines, the sequences of generators where each stage processes data and passes it along to the next.

In [20]:
def read_file(filename):
    for line in open(filename, 'r'):
        yield line.strip()


def filter_lines(lines):
    for line in lines:
        if not line.startswith('#'): # skip comments
            yield line


def transform_lines(lines):
    for line in lines:
        yield line.upper()


# Chaining generators together
pipeline = transform_lines(filter_lines(read_file("sample\\data.txt")))
for processed_line in pipeline:
    print(processed_line)

APPLE
BANANA
CHERRY
DATE
ELDERBERRY
FIG
GRAPE
HONEYDEW
INDIAN FIG
JACKFRUIT
KIWI
LEMON
MANGO
NECTARINE
ORANGE
PAPAYA
QUINCE
RASPBERRY
STRAWBERRY
TANGERINE
UGLI FRUIT
VANILLA
WATERMELON
XIGUA
YAM
ZUCCHINI
ALPHA
BETA
GAMMA
DELTA
EPSILON
THETA
LAMBDA
OMEGA
LINE FORTY ONE
LINE FORTY TWO
LINE FORTY THREE
LINE FORTY FOUR
LINE FORTY FIVE
LINE FORTY SIX
LINE FORTY SEVEN
LINE FORTY EIGHT
LINE FORTY NINE
LINE FIFTY


Here, each generator acts like a stage in a data refinery where, one reads the data, another filters it, and the next transforms it.

## 11.8 Function Introspection & Metadata

Python functions are far more than an executable code blocks, they are first-class objects enriched with a wealth of metadata that encapsulates their behavior, structure, and semantic intent.

#### Basic Function Introspection using built-in Attributes

Every Python function is is a comprehensive object enriched with intrinsic metadata that defines its structure, context, and behavior. Here’s a simple example:

In [21]:
def greet(name: str = "Guest") -> str:
    """Returns a greeting message."""
    return f"Hello, {name}!"

print(greet.__name__) # greet
print(greet.__annotations__) # {'name': <class 'str'>, 'return': <class 'str'>}
print(greet.__doc__) # Returns a greeting message.
print(greet.__defaults__) # ('Guest',)

greet
{'name': <class 'str'>, 'return': <class 'str'>}
Returns a greeting message.
('Guest',)


#### The inspect Module

While the built-in attributes of a Python function provide an essential foundation for introspection, the inspect module elevates this capability to a far more comprehensive and analytical level. For instance:

In [22]:
import inspect

def square(x):
    return x * x

print(inspect.signature(square)) # (x)
print(inspect.getsource(square)) # returns the source code
print(inspect.getmodule(square)) # <module '_main_'>

(x)
def square(x):
    return x * x

<module '__main__'>


#### Runtime Introspection using Built-in Functions

Python provides a collection of built-in functions that enable lightweight and immediate introspection at runtime, eliminating the need for external modules. For example:

In [23]:
def multiply(a, b):
    """Multiplies two numbers."""
    return a * b

print(dir(multiply)) # lists all attributes of the function
print(callable(multiply)) # True
help(multiply) # Displays function info

['__annotations__', '__builtins__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__getstate__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__type_params__']
True
Help on function multiply in module __main__:

multiply(a, b)
    Multiplies two numbers.



## Code Object Introspection via __code__

Every function in Python is underpinned by an internal code object, which encapsulates the low-level structural and operational details of its implementation. For example

In [ ]:
def divide(a, b):
    return a / b

print(divide.__code__.co_varnames) # ('a', 'b')
print(divide.__code__.co_filename) # path to the source file
print(divide.__code__.co_firstlineno)

('a', 'b')
1


## 11.9 Functional Programming Paradigm

The Functional Programming (FP) paradigm is a declarative style of programming that focuses on what to solve rather than how to solve it. Refer the book section 11.9 for more details...

## 11.10 Functional Approach to Data Transformation

Python embraces a functional approach to data transformation through a triad of powerful functional constructs: map(), filter(), and reduce()

#### The map() Function - Transforming Data

The map() function in Python represents the essence of functional data transformation by applying a specified function to every element within an iterable such as a list, tuple, or set, without modifying the original data structure.

In [26]:
numbers = [1, 2, 3, 4, 5]
squares = map(lambda x: x ** 2, numbers)
print(list(squares)) # [1, 4, 9, 16, 25]

[1, 4, 9, 16, 25]


#### The filter() Function – Selecting Data

The filter() function in Python serves as a logical sieve that refines an existing dataset by including only those elements that meet a specified condition

In [27]:
numbers = [10, 15, 20, 25, 30]
even_numbers = filter(lambda x: x % 2 == 0, numbers)
print(list(even_numbers)) # [10, 20, 30]

[10, 20, 30]


#### The reduce() Function – Combining Data

The reduce() function, available in Python’s functools module, embodies the principle of cumulative computation.

In [28]:
from functools import reduce

numbers = [1, 2, 3, 4, 5]
product = reduce(lambda a, b: a * b, numbers)
print(product) # 120

120


## 11.11 Functools

The functools module is one of Python’s more powerful yet often overlooked standard libraries. As its name suggests, it is designed to enhance Python’s expressive capabilities by offering utilities that make functions more flexible, reusable, and efficient.

#### functools.wraps() — Preserving Function Identity

When you create decorators, the wrapper function usually hides the metadata (like _name_, _doc_) of the original function. functools.wraps() is a decorator designed specifically to solve this

In [30]:
from functools import wraps

def logger(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with {args} and {kwargs}")
        return func(*args, **kwargs)
    return wrapper


@logger
def greet(name):
    """Greets a user by name."""
    return f"Hello, {name}!"


print(greet("Ivaan"))
print(greet.__name__) # greet
print(greet.__doc__) # Greets a user by name.

Calling greet with ('Ivaan',) and {}
Hello, Ivaan!
greet
Greets a user by name.


#### functools.lru_cache() — Speed Through Caching

When a function performs heavy computation or repeatedly processes the same inputs, recomputing results each time wastes resources. The Least Recently Used (LRU) cache decorator solves this by storing previously computed results in memory and reusing them when the same inputs reappear.

In [31]:
from functools import lru_cache
import time

@lru_cache(maxsize=None)
def slow_square(n):
    time.sleep(1) # simulate slow operation
    return n * n

print(slow_square(4)) # Takes ~1 second
print(slow_square(4)) # Instant! (cached)

16
16


#### functools.partial() — Pre-Filling Function Arguments

partial() is a gem for simplifying repetitive calls. It allows you to “freeze” or pre-fill some of a function’s arguments, creating a new function with fewer parameters

In [32]:
from functools import partial
def multiply(a, b):
    return a * b

double = partial(multiply, 2)
triple = partial(multiply, 3)
print(double(5)) # 10
print(triple(4)) # 12

10
12


#### functools.singledispatch() — Function Overloading Made Simple

Python doesn’t support traditional function overloading by default, but singledispatch provides a way to define generic functions that behave differently depending on the type of their first argument.

In [34]:
from functools import singledispatch

@singledispatch
def process(value):
    print(f"Default: {value}")

@process.register(int)
def _(value):
    print(f"Processing integer: {value * 2}")

@process.register(list)
def _(value):
    print(f"Processing list of length {len(value)}")


process(5) # Processing integer: 10
process([1, 2]) # Processing list of length 2
process("Hi") # Default: Hi

Processing integer: 10
Processing list of length 2
Default: Hi


#### functools.total_ordering() — Simplifying Class Comparisons

When defining custom classes, implementing all comparison methods (<, >, <=, >=) can be repetitive. The @total_ordering decorator allows you to define just one or two, and Python will infer the rest.

In [35]:
from functools import total_ordering

@total_ordering
class Book:
    def __init__(self, title, pages):
        self.title = title
        self.pages = pages

    def __eq__(self, other):
        return self.pages == other.pages
    
    def __lt__(self, other):
        return self.pages < other.pages
    

book1 = Book("Python 101", 300)
book2 = Book("Advanced Python", 500)
print(book1 < book2) # True
print(book1 >= book2) # False

True
False
